# Importing Data Set 

In [ ]:
import tensorflow as tf 
import pandas as pd

In [ ]:
dataset = pd.read_csv('pima-indians-diabetes.csv')

In [61]:
dataset.head()

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class
0,6,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1
1,1,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0
2,8,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1
3,1,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0
4,0,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1


# Normalizing Data

In [ ]:
norm_col =  ['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']
# Creating the list of the columns to be normalised

In [ ]:
dataset[norm_col] = dataset[norm_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))  #performing normalisation

In [60]:
dataset.head()  # You can see the data se Normalised 

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class
0,6,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1
1,1,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0
2,8,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1
3,1,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0
4,0,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1


# Creating Feature Columns

## Creating feature columns for continous data

In [ ]:
preg = tf.feature_column.numeric_column('Number_pregnant')
gluc = tf.feature_column.numeric_column('Glucose_concentration')
press = tf.feature_column.numeric_column('Blood_pressure')
tri = tf.feature_column.numeric_column('Triceps')
insulin = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
pedigree = tf.feature_column.numeric_column('Pedigree')
age = tf.feature_column.numeric_column('Age')

## Creating Age buckets to divide age into different categories

In [ ]:
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[20,30,40,50,60,70,80])

## Putting feature column together

In [ ]:
feat_cols = [preg ,gluc,press ,tri ,insulin,bmi,pedigree, age_buckets]


# Train Test Split 

In [ ]:
x_data = dataset.drop('Class',axis=1) # Removing the class column as it is part of the y_data

In [ ]:
labels = dataset['Class'] #labels wil used as y_data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.33, random_state=101)

# Creating input function 

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)
#Input function is used to input the values to the model for prediction

# Creating model

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

# Training Model

In [ ]:
model.train(input_fn=input_func,steps=1000)

# Evaluating Model

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [ ]:
results = model.evaluate(eval_input_func)

In [59]:
results

{'accuracy': 0.72047246,
 'accuracy_baseline': 0.65748036,
 'auc': 0.78742516,
 'auc_precision_recall': 0.635725,
 'average_loss': 0.5326862,
 'label/mean': 0.34251967,
 'loss': 5.203934,
 'precision': 0.5952381,
 'prediction/mean': 0.38067704,
 'recall': 0.57471263,
 'global_step': 1000}